# Sprint15課題 ディープラーニングワークフレーム2

## 2.公式Example

### 【問題1】公式Exampleを分担して実行

In [0]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

In [0]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [0]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [0]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [0]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [0]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

Instructions for updating:
Colocations handled automatically by placer.
F
i
r
s
t


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [0]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [0]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([ 5, 37, 60, 55, 24, 20, 16, 49,  2,  8, 59, 28, 38, 37, 37, 22,  7,
       32, 10, 57, 46, 15, 17,  7, 12, 32, 13, 17, 16, 30,  6, 30, 47, 39,
       51, 41, 62, 62, 19, 38, 17, 64, 36, 14, 19,  6, 53, 64, 49, 13, 34,
       49, 32, 37,  7, 36, 12, 20, 57,  7, 37, 55, 21, 40, 59, 41, 41, 12,
       36,  1, 21, 23, 59,  9, 25, 57, 35,  7, 44, 64,  7, 64, 15, 52, 22,
       36, 54, 27, 33, 30, 38, 45, 62,  6, 30, 61, 64, 44, 50,  2])

In [0]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "ou mercy; you are the singer: I will say\nfor you. It is 'music with her silver sound,'\nbecause music"

Next Char Predictions: 
 "'YvqLHDk!.uPZYYJ-T:shCE-?TAEDR,RiamcxxGZEzXBG,ozkAVkTY-X?Hs-YqIbucc?X IKu3MsW-fz-zCnJXpOURZgx,Rwzfl!"


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1756463


In [0]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=3

In [0]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/3
173/174 [============================>.] - ETA: 0s - loss: 2.6913WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/network.py:1436: update_checkpoint_state (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
174/174 [==============================] - 26s 147ms/step - loss: 2.6882
Epoch 2/3
174/174 [==============================] - 23s 135ms/step - loss: 1.9497
Epoch 3/3
174/174 [==============================] - 23s 135ms/step - loss: 1.6814


In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_3'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"ROMEO: "))

Instructions for updating:
Use tf.random.categorical instead.
ROMEO: MISTAR:
Pons thou artonce to in the rize, bother'd broke
YorE but not ansulder'd: a prayep one

VALSCANT:
Why has and helbs thoughter, orname, and, pped, she's one controught
Firsumen so father laughter honour.

DUGHES:
O, thoughsort you madce deture sools!

First Sendor:
His we on cloudder of wither him to our deep, and supsect and in armosting.

LADY ANNE:
Not with man illoks him here?

FLARANUS:
Fair, think our ilsoral provan: grees, then
That leigher'd plovent,
And Ricann Juttress their complece, hol chorse your hand aponets mackendly you.

OEFORD:
Cave'd thangey: what's in death with yours the
Kature thy toobither criet;
Which yes spoth in three wound never
Yet my speforall, ere and repoven;
You have take to be hen farther such on't.
Think, we shall be juttlewell'd even your?

PROMPROME:
By you proded and bling you
no worred in pare cry:
And come freaty, if my sear, may you. Fir, Where eveny more.
Have now in din

## 3.異なるフレームワークへの書き換え

### 【問題2】Iris（2値分類）をKerasで学習

In [8]:
"""
Kerasで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)


#Functional APIでのモデル構築
input_data = tf.keras.layers.Input(shape=(X_train.shape[1],)) # 入力層
output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(input_data) # 出力層

model = tf.keras.Model(inputs=input_data, outputs=output)

model.summary()
model.compile(loss='binary_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])
history = model.fit(X_train, y_train,
                    batch_size=10,
                    epochs=500,
                    verbose=1,
                    validation_data=(X_val, y_val))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 4)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________
Train on 64 samples, validate on 16 samples
Epoch 1/500
64/64 [==============================] - 1s 19ms/step - loss: 4.7404 - acc: 0.4688 - val_loss: 3.0859 - val_acc: 0.6250
Epoch 2/500
64/64 [==============================] - 0s 515us/step - loss: 4.0751 - acc: 0.4687 - val_loss: 2.6376 - val_acc: 0.6250
Epoch 3/500
64/64 [==============================] - 0s 618us/step - loss: 3.4502 - acc: 0.4688 - val_loss: 2.1881 - val_acc: 0.6250
Epoch 4/500
64/64 [==============================] - 0s 556us/step - loss: 2.8345 - acc: 0.4688 - val_

64/64 [==============================] - 0s 425us/step - loss: 0.4730 - acc: 0.8906 - val_loss: 0.4759 - val_acc: 0.9375
Epoch 119/500
64/64 [==============================] - 0s 473us/step - loss: 0.4710 - acc: 0.8750 - val_loss: 0.4684 - val_acc: 0.9375
Epoch 120/500
64/64 [==============================] - 0s 450us/step - loss: 0.4691 - acc: 0.8906 - val_loss: 0.4732 - val_acc: 0.9375
Epoch 121/500
64/64 [==============================] - 0s 482us/step - loss: 0.4678 - acc: 0.8750 - val_loss: 0.4737 - val_acc: 0.9375
Epoch 122/500
64/64 [==============================] - 0s 449us/step - loss: 0.4649 - acc: 0.8750 - val_loss: 0.4583 - val_acc: 1.0000
Epoch 123/500
64/64 [==============================] - 0s 463us/step - loss: 0.4644 - acc: 0.8906 - val_loss: 0.4487 - val_acc: 1.0000
Epoch 124/500
64/64 [==============================] - 0s 466us/step - loss: 0.4592 - acc: 0.9219 - val_loss: 0.4493 - val_acc: 1.0000
Epoch 125/500
64/64 [==============================] - 0s 386us/step 

Epoch 179/500
64/64 [==============================] - 0s 407us/step - loss: 0.3616 - acc: 0.9531 - val_loss: 0.3492 - val_acc: 1.0000
Epoch 180/500
64/64 [==============================] - 0s 513us/step - loss: 0.3621 - acc: 0.9531 - val_loss: 0.3416 - val_acc: 1.0000
Epoch 181/500
64/64 [==============================] - 0s 507us/step - loss: 0.3583 - acc: 0.9531 - val_loss: 0.3431 - val_acc: 1.0000
Epoch 182/500
64/64 [==============================] - 0s 450us/step - loss: 0.3574 - acc: 0.9531 - val_loss: 0.3428 - val_acc: 1.0000
Epoch 183/500
64/64 [==============================] - 0s 476us/step - loss: 0.3555 - acc: 0.9531 - val_loss: 0.3382 - val_acc: 1.0000
Epoch 184/500
64/64 [==============================] - 0s 444us/step - loss: 0.3558 - acc: 0.9219 - val_loss: 0.3271 - val_acc: 1.0000
Epoch 185/500
64/64 [==============================] - 0s 413us/step - loss: 0.3540 - acc: 0.9375 - val_loss: 0.3261 - val_acc: 1.0000
Epoch 186/500
64/64 [==============================] - 

Epoch 240/500
64/64 [==============================] - 0s 530us/step - loss: 0.2902 - acc: 0.9688 - val_loss: 0.2611 - val_acc: 1.0000
Epoch 241/500
64/64 [==============================] - 0s 498us/step - loss: 0.2908 - acc: 0.9688 - val_loss: 0.2672 - val_acc: 1.0000
Epoch 242/500
64/64 [==============================] - 0s 536us/step - loss: 0.2882 - acc: 0.9531 - val_loss: 0.2644 - val_acc: 1.0000
Epoch 243/500
64/64 [==============================] - 0s 718us/step - loss: 0.2867 - acc: 0.9687 - val_loss: 0.2596 - val_acc: 1.0000
Epoch 244/500
64/64 [==============================] - 0s 744us/step - loss: 0.2894 - acc: 0.9531 - val_loss: 0.2502 - val_acc: 1.0000
Epoch 245/500
64/64 [==============================] - 0s 588us/step - loss: 0.2857 - acc: 0.9688 - val_loss: 0.2548 - val_acc: 1.0000
Epoch 246/500
64/64 [==============================] - 0s 608us/step - loss: 0.2843 - acc: 0.9688 - val_loss: 0.2577 - val_acc: 1.0000
Epoch 247/500
64/64 [==============================] - 

Epoch 301/500
64/64 [==============================] - 0s 542us/step - loss: 0.2419 - acc: 0.9687 - val_loss: 0.2065 - val_acc: 1.0000
Epoch 302/500
64/64 [==============================] - 0s 462us/step - loss: 0.2418 - acc: 0.9687 - val_loss: 0.2031 - val_acc: 1.0000
Epoch 303/500
64/64 [==============================] - 0s 555us/step - loss: 0.2410 - acc: 0.9688 - val_loss: 0.2050 - val_acc: 1.0000
Epoch 304/500
64/64 [==============================] - 0s 423us/step - loss: 0.2404 - acc: 0.9844 - val_loss: 0.2077 - val_acc: 1.0000
Epoch 305/500
64/64 [==============================] - 0s 442us/step - loss: 0.2390 - acc: 0.9844 - val_loss: 0.2076 - val_acc: 1.0000
Epoch 306/500
64/64 [==============================] - 0s 558us/step - loss: 0.2386 - acc: 0.9844 - val_loss: 0.2068 - val_acc: 1.0000
Epoch 307/500
64/64 [==============================] - 0s 634us/step - loss: 0.2390 - acc: 0.9844 - val_loss: 0.2035 - val_acc: 1.0000
Epoch 308/500
64/64 [==============================] - 

Epoch 362/500
64/64 [==============================] - 0s 532us/step - loss: 0.2086 - acc: 0.9844 - val_loss: 0.1721 - val_acc: 1.0000
Epoch 363/500
64/64 [==============================] - 0s 470us/step - loss: 0.2078 - acc: 0.9844 - val_loss: 0.1751 - val_acc: 1.0000
Epoch 364/500
64/64 [==============================] - 0s 478us/step - loss: 0.2074 - acc: 0.9687 - val_loss: 0.1730 - val_acc: 1.0000
Epoch 365/500
64/64 [==============================] - 0s 543us/step - loss: 0.2079 - acc: 0.9687 - val_loss: 0.1680 - val_acc: 1.0000
Epoch 366/500
64/64 [==============================] - 0s 579us/step - loss: 0.2067 - acc: 0.9844 - val_loss: 0.1695 - val_acc: 1.0000
Epoch 367/500
64/64 [==============================] - 0s 517us/step - loss: 0.2064 - acc: 0.9687 - val_loss: 0.1724 - val_acc: 1.0000
Epoch 368/500
64/64 [==============================] - 0s 496us/step - loss: 0.2054 - acc: 0.9687 - val_loss: 0.1711 - val_acc: 1.0000
Epoch 369/500
64/64 [==============================] - 

Epoch 423/500
64/64 [==============================] - 0s 523us/step - loss: 0.1834 - acc: 0.9687 - val_loss: 0.1472 - val_acc: 1.0000
Epoch 424/500
64/64 [==============================] - 0s 503us/step - loss: 0.1826 - acc: 0.9687 - val_loss: 0.1450 - val_acc: 1.0000
Epoch 425/500
64/64 [==============================] - 0s 556us/step - loss: 0.1830 - acc: 0.9687 - val_loss: 0.1423 - val_acc: 1.0000
Epoch 426/500
64/64 [==============================] - 0s 572us/step - loss: 0.1824 - acc: 0.9687 - val_loss: 0.1437 - val_acc: 1.0000
Epoch 427/500
64/64 [==============================] - 0s 498us/step - loss: 0.1880 - acc: 0.9687 - val_loss: 0.1506 - val_acc: 1.0000
Epoch 428/500
64/64 [==============================] - 0s 560us/step - loss: 0.1817 - acc: 0.9687 - val_loss: 0.1435 - val_acc: 1.0000
Epoch 429/500
64/64 [==============================] - 0s 529us/step - loss: 0.1813 - acc: 0.9844 - val_loss: 0.1411 - val_acc: 1.0000
Epoch 430/500
64/64 [==============================] - 

Epoch 484/500
64/64 [==============================] - 0s 533us/step - loss: 0.1642 - acc: 0.9687 - val_loss: 0.1302 - val_acc: 1.0000
Epoch 485/500
64/64 [==============================] - 0s 443us/step - loss: 0.1648 - acc: 0.9687 - val_loss: 0.1308 - val_acc: 1.0000
Epoch 486/500
64/64 [==============================] - 0s 550us/step - loss: 0.1664 - acc: 0.9687 - val_loss: 0.1256 - val_acc: 1.0000
Epoch 487/500
64/64 [==============================] - 0s 564us/step - loss: 0.1635 - acc: 0.9687 - val_loss: 0.1305 - val_acc: 1.0000
Epoch 488/500
64/64 [==============================] - 0s 547us/step - loss: 0.1636 - acc: 0.9687 - val_loss: 0.1331 - val_acc: 1.0000
Epoch 489/500
64/64 [==============================] - 0s 567us/step - loss: 0.1648 - acc: 0.9687 - val_loss: 0.1304 - val_acc: 1.0000
Epoch 490/500
64/64 [==============================] - 0s 567us/step - loss: 0.1665 - acc: 0.9687 - val_loss: 0.1328 - val_acc: 1.0000
Epoch 491/500
64/64 [==============================] - 

In [9]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', score[1])


Test accuracy: 0.8999999761581421


### 【問題3】Iris（多値分類）をKerasで学習

In [10]:
"""
Kerasで実装したニューラルネットワークを使いIrisデータセットをクラス分類する
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_one_hot = enc.fit_transform(y[:, np.newaxis])

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

#Functional APIでのモデル構築
input_data = tf.keras.layers.Input(shape=(X_train.shape[1],)) # 入力層
output = tf.keras.layers.Dense(y_train.shape[1], activation=tf.nn.sigmoid)(input_data) # 出力層

model = tf.keras.Model(inputs=input_data, outputs=output)

model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])
history = model.fit(X_train, y_train,
                    batch_size=10,
                    epochs=100,
                    verbose=1,
                    validation_data=(X_val, y_val))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 4)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 15        
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________
Train on 96 samples, validate on 24 samples
Epoch 1/100
96/96 [==============================] - 1s 13ms/step - loss: 1.8235 - acc: 0.3646 - val_loss: 1.4579 - val_acc: 0.3750
Epoch 2/100
96/96 [==============================] - 0s 372us/step - loss: 1.3895 - acc: 0.3646 - val_loss: 1.1081 - val_acc: 0.3750
Epoch 3/100
96/96 [==============================] - 0s 627us/step - loss: 1.1138 - acc: 0.3646 - val_loss: 0.9446 - val_acc: 0.3750
Epoch 4/100
96/96 [==============================] - 0s 570us/step - loss: 0.9861 - acc: 0.3333 - va

In [12]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', score[1])


Test accuracy: 1.0


### 【問題4】House PricesをKerasで学習

In [13]:
"""
Kerasで実装したニューラルネットワークを使いHouse Pricesデータセットを価格予測する
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

# データセットの読み込み
dataset_path ="train.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]
y = np.log(np.array(y))
X = np.array(X)
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

#標準化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

#Functional APIでのモデル構築
input_data = tf.keras.layers.Input(shape=(X_train.shape[1],)) # 入力層
output = tf.keras.layers.Dense(y.shape[1], activation=tf.nn.relu)(input_data) # 出力層

model = tf.keras.Model(inputs=input_data, outputs=output)

model.summary()
model.compile(loss='mean_squared_error',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['mae'])
history = model.fit(X_train, y_train,
                    batch_size=10,
                    epochs=20,
                    verbose=1,
                    validation_data=(X_val, y_val))

/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/clouds_kimura/anaconda3/envs/pythonml/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________
Train on 934 samples, validate on 234 samples
Epoch 1/20
934/934 [==============================] - 1s 1ms/step - loss: 115.1131 - mean_absolute_error: 10.6422 - val_loss: 106.0927 - val_mean_absolute_error: 10.1654
Epoch 2/20
934/934 [==============================] - 0s 293us/step - loss: 100.0670 - mean_absolute_error: 9.8304 - val_loss: 91.3259 - val_mean_absolute_error: 9.3121
Epoch 3/20
934/934 [==============================] - 0s 268us/step - loss: 84.5806 - mean_absolute_error: 8.9586 - val_loss: 76.2537 - val_mean_absolute_error

In [14]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test mae:', score[1])


Test mae: 0.3903266632393615


### 【問題5】MNISTをKerasで学習

In [15]:
"""
Kerasで実装したニューラルネットワークを使いIrisデータセットをクラス分類する
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

"""
使用するデータセットを用意
"""
from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)

#Functional APIでのモデル構築
input_data = tf.keras.layers.Input(shape=(X_train.shape[1],)) # 入力層
output = tf.keras.layers.Dense(y_train.shape[1], activation=tf.nn.sigmoid)(input_data) # 出力層

model = tf.keras.Model(inputs=input_data, outputs=output)

model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])
history = model.fit(X_train, y_train,
                    batch_size=10,
                    epochs=5,
                    verbose=1,
                    validation_data=(X_val, y_val))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________
Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 17s 344us/step - loss: 0.4434 - acc: 0.8849 - val_loss: 0.4296 - val_acc: 0.8992
Epoch 2/5
48000/48000 [==============================] - 17s 346us/step - loss: 0.4094 - acc: 0.9027 - val_loss: 0.4150 - val_acc: 0.9086
Epoch 3/5
48000/48000 [==============================] - 21s 431us/step - loss: 0.4052 - acc: 0.9044 - val_loss: 0.4504 - val_acc: 0.9070
Epoch 4/5
48000/48000 [==============================] - 21s 439us/step 

In [19]:
score = model.evaluate(X_test, y_test_one_hot, verbose=0)
print('Test accuracy:', score[1])


Test accuracy: 0.91
